# Deteksi Kecurangan pada Sistem *Checkout* Mandiri di Toko Ritel
## *- Fraud detection at Self-Checkouts in Retail -*

Kami dari **Kelompok 01** yang beranggotakan:

* NIM. 2001123 - Muhamad Fikri Nur Bakhtiar
* NIM. 2005128 - Muhammad Satria Ramadhani
* NIM. 2009085 - Wildan Septi Ramadhan

Mengerjakan tugas Klasifikasi pada mata kuliah **Data Mining & Warehouse** dengan menggunakan dataset **"*Fraud Detection at Self-Checkouts in Retail*"**. Adapun sumber dataset dapat diakses melalui:
**https://www.data-mining-cup.com/reviews/dmc-2019/**

## Dokumentasi.

### Deskripsi dataset.
Akhir-akhir ini, jumlah stasiun yang menggunakan sistem pembayaran mandiri (*self-checkouts station*) terus meningkat. Stasiun ini terbagi ke dalam dua bagian, yakni:

* *Stationary self-checkouts*, di mana pelanggan membawa keranjang mereka ke tempat pemindaian abrang dan membayar produk mereka.
* *Semi-stationary self-checkouts*, di mana pelanggan memindai langsung produk mereka, kemudian membayarnya di loket pembayaran. Di sini, toko menyediakan alat pemindainya atau pelanggan menggunakan *smartphone* masing-masing.

Tentu saja, proses otomatis ini mampu menghilangkan antrean yang panjang dan mempercepat proses pembayaran. Akan tetapi, bagaimana *retailer* mencegah penyalahgunaan yang bisa dilakukan oleh pelanggan? Bagaimana mereka memutuskan pelanggan mana yang harus diperiksa tanpa mengganggu kenyamanan pelanggan lain?

Tujuan model klasifikasi ini adalah untuk mendeteksi, apakah seseorang mencurangi belanjaannya atau tidak.

### Keterangan parameter.
* **trustLevel** - Tingkat kepercayaan pelanggan, dalam bentuk bilangan bulat (1 = terendah, 6 = tertinggi).
* **totalScanTimeinSeconds** - Total waktu pemindaian antara produk pertama hingga produk terakhir, dalam bentuk bilangan bulat (detik).
* **grandTotal** - Total besar produk yang dipindai, dalam bentuk bilangan desimal.
* **lineItemVoids** - Jumlah pemindaian yang dibatalkan, dalam bentuk bilangan bulat.
* **scanWithoutRegistration** - Jumlah percobaan untuk menyalakan alat pemindai tanpa pemindai apapun, dalam bentuk bilangan bulat.
* **quantityModification** - Jumlah produk yang dipindai yang kuantitasnya dimodifikasi, dalam bentuk bilangan bulat.
* **scannedLineItemsPerSecond** - Rata-rata jumlah produk dipindai setiap detiknya, dalam bentuk bilangan bulat.
* **valuePerSecond** - Rata-rata nilai produk yang dipindai setiap detiknya, dalam bentuk bilangan desimal.
* **lineItemVoidsPerPosition** -
* ***fraud*** - Hasil klasifikasi, apakah pelanggan melakukan kecurangan (1 / fraud) atau tidak (0 / non-fraud).

## Inisialisasi pustaka umum dan dataset ke Jupyter Notebook.
Bagian ini bisa diabaikan.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
# Direktori file menyesuaikan.
df = pd.read_csv("D:\\University\\Semester 03\\Data Mining and Warehouse\\Klasifikasi\\train.csv", sep = '|')
df

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379,0
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143,0
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769,0
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862,0
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111,0
...,...,...,...,...,...,...,...,...,...,...
1874,1,321,76.03,8,7,2,0.071651,0.236854,0.347826,0
1875,1,397,41.89,5,5,0,0.065491,0.105516,0.192308,1
1876,4,316,41.83,5,8,1,0.094937,0.132373,0.166667,0
1877,2,685,62.68,1,6,2,0.035036,0.091504,0.041667,0


## Praproses.
Bagian ini akan memeriksa tipe data, mengubah nilai NaN, dan membersihkan data.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1879 entries, 0 to 1878
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   trustLevel                 1879 non-null   int64  
 1   totalScanTimeInSeconds     1879 non-null   int64  
 2   grandTotal                 1879 non-null   float64
 3   lineItemVoids              1879 non-null   int64  
 4   scansWithoutRegistration   1879 non-null   int64  
 5   quantityModifications      1879 non-null   int64  
 6   scannedLineItemsPerSecond  1879 non-null   float64
 7   valuePerSecond             1879 non-null   float64
 8   lineItemVoidsPerPosition   1879 non-null   float64
 9   fraud                      1879 non-null   int64  
dtypes: float64(4), int64(6)
memory usage: 146.9 KB


Semua data berbentuk numerik (int/float). Berdasarkan deskripsi parameter di atas, kami menyimpulkan bahwa semua data penting.

In [4]:
df.isnull().sum()

trustLevel                   0
totalScanTimeInSeconds       0
grandTotal                   0
lineItemVoids                0
scansWithoutRegistration     0
quantityModifications        0
scannedLineItemsPerSecond    0
valuePerSecond               0
lineItemVoidsPerPosition     0
fraud                        0
dtype: int64

Tidak ada data NaN, yang mana kami menyimpulkan data sudah bersih.

## Pembelajaran dan Evaluasi.
Bagian utama dari proses klasifikasi. Mesin akan "mempelajari" dataset menggunakan beberapa model klasifikasi dan membuat prediksi berdasarkan hasil pembelajaran tersebut.

Ada 4 model klasifikasi yang digunakan:

* Bayessian (Naive Bayes).
* Decision Tree.
* Random Forest.
* XGBoost.

Selain itu, semua klasifikasi mempunyai proses yang sama:

* Inisialisasi model/algoritma pembelajaran.
* Proses pembelajaran atau klasifikasi.
* Menyimpan hasil prediksi dari data Test.
* Menyimpan nilai akurasi.
* Keluaran, berupa akurasi, hasil klasifikasi, dan confusion matrix.

### Inisialisasi pustaka algoritma untuk klasifikasi.
Bagian ini bisa diabaikan.

In [5]:
# Split Dataset.
from sklearn.model_selection import train_test_split
import pickle

# General.
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

# Naive Bayes.
from sklearn.naive_bayes import GaussianNB

# Decision Tree & Random Forest.
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier

# XGBoost.
import xgboost as xgb

# Save model.
import joblib

### Split dataset.
Bagian ini memisahkan antara data untuk *training* (train) dan untuk *testing* (test).

* **X** - *Predictor*, atau kumpulan atribut yang akan dijadikan *feature*. X diambil dari semua atribut yang mempengaruhi **fraud**.
* **Y** - *Response*, atau label yang akan dijadikan model prediksi. Y diambil dari data **fraud**.
* **X_Train** - Data X untuk *training*.
* **Y_Train** - Data Y untuk *training*.
* **X_Test** - Data X untuk *testing*.
* **Y_Test** - Data Y untuk *testing*.

#### Memisahkan X dan Y.

In [6]:
X = df.drop("fraud", axis = 1) # Mengambil semua atribut, kecuali fraud.
X                              # Memastikan atribut fraud sudah hilang.

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition
0,5,1054,54.70,7,0,3,0.027514,0.051898,0.241379
1,3,108,27.36,5,2,4,0.129630,0.253333,0.357143
2,3,1516,62.16,3,10,5,0.008575,0.041003,0.230769
3,6,1791,92.31,8,4,4,0.016192,0.051541,0.275862
4,5,430,81.53,3,7,2,0.062791,0.189605,0.111111
...,...,...,...,...,...,...,...,...,...
1874,1,321,76.03,8,7,2,0.071651,0.236854,0.347826
1875,1,397,41.89,5,5,0,0.065491,0.105516,0.192308
1876,4,316,41.83,5,8,1,0.094937,0.132373,0.166667
1877,2,685,62.68,1,6,2,0.035036,0.091504,0.041667


In [7]:
Y = df.fraud # Mengambil atribut fraud.
Y            # Memastikan hanya fraud yang diambil atributnya.

0       0
1       0
2       0
3       0
4       0
       ..
1874    0
1875    1
1876    0
1877    0
1878    0
Name: fraud, Length: 1879, dtype: int64

#### Memisahkan X_Train, X_Test, Y_Train, dan Y_Test.

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 123)

# Menyimpan nama kolom untuk keperluan prediksi.
with open('D:\\University\\Semester 03\\Data Mining and Warehouse\\Klasifikasi\\x_train_columns.pickle', 'wb') as fp:
    pickle.dump(X_train.columns, fp)

### Klasifikasi Bayessian.

In [9]:
# Proses.
clf_bayes = GaussianNB()                     
clf_bayes.fit(X_train, Y_train)
Y_pred = clf_bayes.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)

# Keluaran.
print("Accuracy : {}".format(acc))
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

Accuracy : 0.8670212765957447
              precision    recall  f1-score   support

           0       1.00      0.86      0.93       361
           1       0.23      1.00      0.38        15

    accuracy                           0.87       376
   macro avg       0.62      0.93      0.65       376
weighted avg       0.97      0.87      0.90       376

[[311  50]
 [  0  15]]


Jika dilihat dari hasil tersebut, model Naive Bayes mempunyai akurasi 86%, dengan keterangan berikut:

* Prediksi untuk non-fraud (0) dibilang cukup baik karena hasilnya di atas 80%.
* Prediksi untuk fraud (1) bisa dibilang kurang baik karena banyak yang nilainya di bawah 50%.
* Dalam *confusion matrix* pun, banyak data test yang seharusnya berada di non-fraud malah diprediksi menjadi fraud.

Untuk memperbaiki ini, coba dengan model klasifikasi berikutnya.

### Klasifikasi Decision Tree.

In [10]:
# Proses.
clf_tree = tree.DecisionTreeClassifier()
clf_tree.fit(X_train, Y_train)
Y_pred = clf_tree.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)

# Keluaran.
print("Accuracy : {}".format(acc))
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

Accuracy : 0.976063829787234
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       361
           1       0.75      0.60      0.67        15

    accuracy                           0.98       376
   macro avg       0.87      0.80      0.83       376
weighted avg       0.97      0.98      0.97       376

[[358   3]
 [  6   9]]


Jika dilihat dari hasil tersebut, model Decision Tree mempunyai akurasi 97%, dengan keterangan berikut:

* Akurasi meningkat sekitar 11%, lebih baik daripada model sebelumnya.
* Prediksi untuk non-fraud (0) dibilang sangat baik karena semua nilainya mendekati 100%.
* Prediksi untuk fraud (1) mengalami peningkatan pada precision dan f1-score, tetapi penurunan pada recall. Semua nilai masih di bawah 80%.
* Dalam *confusion matrix*, akurasi data test mengalami peningkatan.

Untuk memperbaikinya lebih lanjut, coba dengan model klasifikasi berikutnya.

### Klasifikasi Random Forest.
Ada beberapa model Random Forest, tergantung seberapa besar n_estimators nya. Di sini, akan ada tiga perbandingan hasil klasifikasi (50, 100, 200).

#### n_estimators = 50.

In [11]:
# Proses.
clf_rforest = RandomForestClassifier(n_estimators = 50, random_state = 123)
clf_rforest.fit(X_train, Y_train)
Y_pred = clf_rforest.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)

# Keluaran.
print("Accuracy : {}".format(acc))
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

Accuracy : 0.9867021276595744
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       361
           1       1.00      0.67      0.80        15

    accuracy                           0.99       376
   macro avg       0.99      0.83      0.90       376
weighted avg       0.99      0.99      0.99       376

[[361   0]
 [  5  10]]


Jika dilihat dari hasil tersebut, model RandomForest dengan n_estimators = 50 mempunyai akurasi 98%, dengan keterangan berikut:

* Akurasi meningkat sekitar 1%.
* Prediksi untuk non-fraud (0) dibilang sangat baik karena semua nilainya benar-benar mendekati 100%.
* Prediksi untuk fraud (1) mengalami peningkatan.
* Dalam *confusion matrix*, akurasi data test mengalami peningkatan. Bahkan tidak ada data non-fraud yang salah prediksi.

Model Random Forest ini sudah cukup baik untuk digunakan dalam test.

#### n_estimators = 100.

In [12]:
# Proses.
clf_rforest = RandomForestClassifier(n_estimators = 100, random_state = 123)
clf_rforest.fit(X_train, Y_train)
Y_pred = clf_rforest.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)

# Keluaran.
print("Accuracy : {}".format(acc))
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

Accuracy : 0.9893617021276596
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       361
           1       1.00      0.73      0.85        15

    accuracy                           0.99       376
   macro avg       0.99      0.87      0.92       376
weighted avg       0.99      0.99      0.99       376

[[361   0]
 [  4  11]]


Jika dilihat dari hasil tersebut, model RandomForest dengan n_estimators = 100 mempunyai akurasi mendekati 98%, dengan keterangan berikut:

* Akurasi meningkat meski tidak signifikan.
* Prediksi untuk fraud (1) mengalami peningkatan.
* Dalam *confusion matrix*, akurasi data test mengalami peningkatan meski tidak signifikan.

Ada satu model Random Forest lagi untuk dilihat.

#### n_estimators = 200.

In [13]:
# Proses.
clf_rforest = RandomForestClassifier(n_estimators = 200, random_state = 123)
clf_rforest.fit(X_train, Y_train)
Y_pred = clf_rforest.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)

# Keluaran.
print("Accuracy : {}".format(acc))
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

Accuracy : 0.9920212765957447
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       361
           1       1.00      0.80      0.89        15

    accuracy                           0.99       376
   macro avg       1.00      0.90      0.94       376
weighted avg       0.99      0.99      0.99       376

[[361   0]
 [  3  12]]


Jika dilihat dari hasil tersebut, model RandomForest dengan n_estimators = 200 mempunyai akurasi 99%, dengan keterangan berikut:

* Akurasi sudah mencapai 99%, tetapi dengan jumlah n_estimators dua kali lipat, hal ini tentunya kurang baik.
* Prediksi untuk fraud (1) mengalami peningkatan.

Di sini, model klasifikasi Random Forest cukup hingga n_estimators = 200.

### Klasifikasi XGBoost.

In [14]:
# Proses.
clf_xgb = xgb.XGBClassifier(use_label_encoder = False, objective = "multi:softprob",
                            num_class = 4, eval_metric = "mlogloss", max_depth = 24,
                            gamma = 0.1, subsample = 0.90, learning_rate = 0.01,
                            n_estimators = 10, nthread = -1)

clf_xgb.fit(X_train, Y_train)
Y_pred = clf_xgb.predict(X_test)
acc = accuracy_score(Y_test, Y_pred)

# Keluaran.
print("Akurasi {}".format(acc))
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test, Y_pred))

Akurasi 0.9920212765957447
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       361
           1       1.00      0.80      0.89        15

    accuracy                           0.99       376
   macro avg       1.00      0.90      0.94       376
weighted avg       0.99      0.99      0.99       376

[[361   0]
 [  3  12]]


Jika dilihat dari hasil tersebut, model XGBoost mempunyai akurasi 99%, dengan keterangan berikut:

* Akurasi sudah mencapai 99% yang menandakan model sangat baik.
* Prediksi untuk fraud (1) mengalami peningkatan.

### Menyimpan Model.
Setelah berbagai macam proses tersebut, model akan disimpan untuk kemudian digunakan sebagai tes prediksi.

In [15]:
# Naive Bayes.
joblib.dump(clf_bayes, "D:\\University\\Semester 03\\Data Mining and Warehouse\\Klasifikasi\\naiveBayes.joblib")

# Decision Tree.
joblib.dump(clf_tree, "D:\\University\\Semester 03\\Data Mining and Warehouse\\Klasifikasi\\decisionTree.joblib")

# Random Forest.
joblib.dump(clf_rforest, "D:\\University\\Semester 03\\Data Mining and Warehouse\\Klasifikasi\\randomForest.joblib")

# XGBoost.
joblib.dump(clf_xgb, "D:\\University\\Semester 03\\Data Mining and Warehouse\\Klasifikasi\\XGBoost.joblib")

['D:\\University\\Semester 03\\Data Mining and Warehouse\\Klasifikasi\\XGBoost.joblib']